In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Regressors
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
#Metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
%matplotlib inline
#Clustering
from sklearn.cluster import KMeans 
from sklearn.cluster import DBSCAN
#Otros
from sklearn.multioutput import MultiOutputRegressor

### CSV Loading

In [6]:
dataset = pd.read_csv('./data/t_data/anime_cleaned_treated_v2_10_b.csv')

In [7]:
dataset.head()

,Genres,Episodes,Decimal,AiredYear,Type,Source,Romance,Shoujo,Ecchi,Shounen,...,Mistery,Thriller,Supernatural,Gore,SciFi,Action,Otro,Members,Score,Favorites
0,"Comedy, Romance, Shounen, Supernatural",12,147,2012.0,5,6,1,0,0,1,...,0,0,1,0,0,0,0,283882,7.63,2809
1,"Comedy, Romance, Shounen",26,19,2007.0,5,6,1,0,0,1,...,0,0,0,0,0,0,0,204003,7.89,2579
2,"Comedy, Romance, Fantasy, Shoujo",51,519,2008.0,5,6,1,1,0,0,...,0,0,0,0,0,0,0,70127,7.55,802
3,"Comedy, Romance, Fantasy",38,7,2002.0,5,9,1,0,0,0,...,0,0,0,0,0,0,0,93312,8.21,3344
4,"Comedy, Romance, Shounen",25,19,2012.0,5,6,1,0,0,1,...,0,0,0,0,0,0,0,182765,8.67,2082


### Training

In [8]:
data = dataset.values
x = data[:,3:23]
Y = data[:,23:]
X = data[:int(len(data)*0.7),3:23]
y = data[:int(len(data)*0.7),23:]
X_t = data[int(len(data)*0.7):,3:23]
y_t = data[int(len(data)*0.7):,23:]
print X[0],y[0]

[2012.0 5 6 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0] [283882 7.63 2809]


In [9]:
DT_R = DecisionTreeRegressor(max_depth=5)
RF_R = RandomForestRegressor(n_estimators=40)
GB_R = MultiOutputRegressor(GradientBoostingRegressor())
BG_R = MultiOutputRegressor(BaggingRegressor(base_estimator=GradientBoostingRegressor()))
#GB_R = GradientBoostingRegressor()
#np.array([6,5,12,147]).reshape(1,-1)

In [10]:
DT_R.fit(X,y)
RF_R.fit(X,y)
GB_R.fit(X,y)
BG_R.fit(X,y)

MultiOutputRegressor(estimator=BaggingRegressor(base_estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_sample...n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
           n_jobs=1)

In [11]:
for i in range(3):
    print "============="
    print RF_R.predict(X_t[i].reshape(1,-1))
    print DT_R.predict(X_t[i].reshape(1,-1))
    print GB_R.predict(X_t[i].reshape(1,-1))
    print BG_R.predict(X_t[i].reshape(1,-1))
    print y_t[i]

[[  8.18362750e+04   7.43250000e+00   7.16200000e+02]]
[[  1.96650178e+05   7.53775862e+00   2.74721839e+03]]
[[  1.14153899e+05   7.48935425e+00   1.40132153e+03]]
[[  1.25774869e+05   7.45760332e+00   1.50701505e+03]]
[699351 8.56 21248]
[[  9.38912500e+03   6.67237500e+00   4.10000000e+01]]
[[ 4359.29951691     6.28183575    24.6763285 ]]
[[ 1759.72128891     6.6086022   -306.1686141 ]]
[[ 4425.03418747     6.6735526   -124.97645457]]
[12381 7.23 108]
[[  2.20615812e+04   6.82516667e+00   4.47354167e+01]]
[[  3.08872201e+04   7.06918239e+00   7.59371069e+01]]
[[  2.13845184e+04   7.05577976e+00   1.16737297e+02]]
[[  2.59513944e+04   7.06910185e+00   3.72984011e+02]]
[4533 6.01 4]


### Results

In [21]:
DT_predicts = DT_R.predict(X_t)
RF_predicts = RF_R.predict(X_t)
GB_predicts = GB_R.predict(X_t)
BG_predicts = BG_R.predict(X_t)
print "RMSE %.3f"% np.sqrt(mean_squared_error(y_t, DT_predicts))
print "RMSE %.3f"% np.sqrt(mean_squared_error(y_t, RF_predicts))
print "RMSE %.3f"% np.sqrt(mean_squared_error(y_t, GB_predicts))
print "RMSE %.3f"% np.sqrt(mean_squared_error(y_t, BG_predicts))

RMSE 56645.444
RMSE 52301.862
RMSE 49177.376
RMSE 47417.649


### Arbitrary new anime prediction test

In [28]:
new_anime = np.array([2020,9,5,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1]).reshape(1,-1)

In [29]:
BG_R.predict(new_anime)

array([[  1.48564065e+05,   7.32543677e+00,   1.92537660e+03]])

### Object persistence

In [30]:
import pickle
with open('./data/objects/regressors.pkl', 'wb') as output:
    pickle.dump(DT_R, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(RF_R, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(GB_R, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(BG_R, output, pickle.HIGHEST_PROTOCOL)